In [1]:
from df.enhance import enhance, init_df, load_audio, save_audio
from pathlib import Path
from tqdm import tqdm
import glob
import os
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
import torch
import soundfile as sf
import pyloudnorm as pyln
from scipy.io.wavfile import write
import librosa


OUTPUT_DIR = Path("train_denoised")

# make the OUTPUT_DIR if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
frcrn = pipeline(Tasks.acoustic_noise_suppression, model='speech_frcrn_ans_cirm_16k')
# Load default model
model, df_state, model_name_suffix = init_df(model_base_dir="DeepFilterNet3/")
# Get our SpeakerID audio
speakerID_audio_folder = "train"
audio_paths = glob.glob(f"{speakerID_audio_folder}/*.wav")
# Enhance each audio
for audio_path in tqdm(audio_paths):
    # get the audio filename without .wav extension
    audio_filename = os.path.splitext(os.path.basename(audio_path))[0]
    output_path = OUTPUT_DIR/f"{audio_filename}_enhanced.wav"
    frcrn(audio_path, output_path=output_path)

    data, rate = sf.read(output_path) # load audio
    # peak normalize audio to -0.1 dB as frcrn tend to output very soft
    peak_normalized_audio = pyln.normalize.peak(data, -0.1)  # not using loudness norm here as it causes a bit of clipping
    sf.write(output_path, peak_normalized_audio, rate)
    if 'PALMTREE' not in audio_path:
        audio, _ = load_audio(output_path, sr=df_state.sr())
        # Denoise the audio
        enhanced = enhance(model, df_state, audio)
        # Save for listening
        save_audio(output_path, enhanced, df_state.sr(), dtype=torch.float16)  # default is torch.int16 which causes clipping on some audios
    audio, sr = librosa.load(output_path, sr=22050)  # downsample to 22.05Khz
    sf.write(output_path, audio, sr)  # save the downsampled one

    data, rate = sf.read(output_path) # load audio
    if 'PALMTREE' not in audio_path:
        # peak normalize audio to -0.1 dB as frcrn tend to output very soft
        normalized_audio = pyln.normalize.peak(data, -0.1)  # not using loudness norm here as it causes a bit of clipping on non palmtree clips
    else:  # PALMTREE ones need louder
        # measure the loudness first 
        meter = pyln.Meter(rate) # create BS.1770 meter
        loudness = meter.integrated_loudness(data)
        # loudness normalize audio to -18 dB LUFS
        normalized_audio = pyln.normalize.loudness(data, loudness, -18.0)
    sf.write(output_path, normalized_audio, rate)
# TOMATOFARMER_memberD_train_enhanced.wav is denosied using DeepfilterNet2 https://huggingface.co/spaces/hshr/DeepFilterNet2 then normalized peak to -0.1db using Adobe Audition


2023-06-13 00:43:35,703 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-06-13 00:43:35,709 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-06-13 00:43:35,727 - modelscope - INFO - Loading done! Current index file version is 1.6.1, with md5 2d7845052188f90f8e56b158b6752809 and a total number of 849 components indexed
/root/venvs/til2023/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-13 00:43:36,812 - modelscope - INFO - initiate model from speech_frcrn_ans_cirm_16k
2023-06-13 00:43:36,814 - modelscope - INFO - initiate model from location speech_frcrn_ans_cirm_16k.
2023-06-13 00:43:36,823 - modelscope - INFO - initialize model from speech_frcrn_ans_cirm_16k
2023-06-13 00:43:37,588 - modelscope - WARNING - No preprocessor field found in cfg.
2023

2023-06-13 00:43:37 | INFO     | DF | Running on torch 2.0.1+cu118
2023-06-13 00:43:37 | INFO     | DF | Running on host Desktop
2023-06-13 00:43:37 | INFO     | DF | Loading model settings of DeepFilterNet3


fatal: not a git repository (or any of the parent directories): .git


2023-06-13 00:43:37 | INFO     | DF | Initializing model `deepfilternet3`
2023-06-13 00:43:39 | INFO     | DF | Found checkpoint DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2023-06-13 00:43:40 | INFO     | DF | Running on device cuda:0
2023-06-13 00:43:40 | INFO     | DF | Model loaded


  0%|          | 0/32 [00:00<?, ?it/s]

...

/root/venvs/til2023/lib/python3.8/site-packages/torchaudio/functional/functional.py:1458: UserWarning: "sinc_interpolation" resampling method name is being deprecated and replaced by "sinc_interp_hann" in the next release. The default behavior remains unchanged.
  warnings.warn(
 94%|█████████▍| 30/32 [00:31<00:01,  1.08it/s]

inputs:(1, 240000)
padding: 24000
inputs after padding:(1, 264000)


/root/venvs/til2023/lib/python3.8/site-packages/torchaudio/functional/functional.py:1458: UserWarning: "sinc_interpolation" resampling method name is being deprecated and replaced by "sinc_interp_hann" in the next release. The default behavior remains unchanged.
  warnings.warn(
 97%|█████████▋| 31/32 [00:32<00:00,  1.04it/s]

inputs:(1, 240000)
padding: 24000
inputs after padding:(1, 264000)


/root/venvs/til2023/lib/python3.8/site-packages/torchaudio/functional/functional.py:1458: UserWarning: "sinc_interpolation" resampling method name is being deprecated and replaced by "sinc_interp_hann" in the next release. The default behavior remains unchanged.
  warnings.warn(
100%|██████████| 32/32 [00:33<00:00,  1.05s/it]
